In [27]:
import pandas as pd
import datetime as dtime
from datetime import datetime
from functions import analysis, feature_engineering, train_test_split, create_model, cv, baseline, \
    test_model, features
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
data = pd.read_csv('/Users/victorialokteva/Downloads/data.csv')
# выкинем колонки, которые не нужы для 1 го задания
data = data.drop(columns=['oaid_hash', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 
                  'g0', 'g1', 'coeff_sum0', 'coeff_sum1'])
data.head()

In [3]:
analysis(data)

Количество записей в таблице:  15821472
соотношние между классами:
1-ый класс: 2.6688351121817235 %, 2-ой класс 97.33116488781828 %

Nan в таблице:
False

Количество уникальных значений для os_id:  11
Количество уникальных значений для country_id:  17
Количество уникальных значений для impressions:  1
Количество уникальных значений для campaign_clicks:  822


In [4]:
# Так как значение этой колоники одинаково для всех строк, уберем ее
data = data.drop(columns=['impressions'])

In [5]:
# Добавим признаков
data = feature_engineering(data)
data.tail()

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,clicks,time,date,time_of_day,is_weekend,month,15_av_clicks_os,15_av_clicks_country,15_av_clicks_month
15821467,2021-10-02 15:51:35,146,530,0,2,9,0,15:51:35,2021-10-02,1,1,10,-1.0,-1.0,-1.0
15821468,2021-09-27 22:03:14,12,22,0,1,6,0,22:03:14,2021-09-27,2,0,9,-1.0,-1.0,-1.0
15821469,2021-10-02 17:41:10,12,1236,0,2,0,0,17:41:10,2021-10-02,1,1,10,-1.0,-1.0,-1.0
15821470,2021-09-29 00:39:32,967,21,0,0,0,0,00:39:32,2021-09-29,3,0,9,-1.0,-1.0,-1.0
15821471,2021-09-28 07:00:18,19,635,0,0,3,0,07:00:18,2021-09-28,0,0,9,-1.0,-1.0,-1.0


In [13]:
X_train, Y_train, X_test, Y_test = train_test_split(data)

In [10]:
model = create_model(X_train, Y_train)
test_model(model,X_test, Y_test )

/Users/victorialokteva/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [16]:
# Подберем параметры регуляризации:
l1 = cv(X_train, Y_train)

KeyboardInterrupt: 

In [ ]:
tuned_model = create_model(X_train, Y_train, l1)
test_model(tuned_model)

In [ ]:
# Сравним с бейзлайном
data = baseline(data)

In [26]:
data.head()

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,impressions,clicks_x,clicks_y
0,2021-09-27 00:01:30.000000,0,0,0,0,0,1,1,0.040687
1,2021-09-30 00:03:59.000000,27,0,0,0,0,1,1,0.040687
2,2021-09-30 00:04:44.000000,27,0,0,0,0,1,1,0.040687
3,2021-09-30 00:05:17.000000,27,0,0,0,0,1,1,0.040687
4,2021-10-01 00:04:29.000000,27,0,0,0,0,1,1,0.040687


In [ ]:
# Данные за последний день
last_day = data['date'].max()
last_day_data = data.loc[data['date']== last_day]
test_model(last_day_data.loc[:, features], last_day_data['clicks'])